# Tutorial 2 - SOLUTION

# Part 1: Data revisited

## Step 1.1 - Load packages and data
As in the previous tutorials, we start by loading the packages and data:

In [1]:
# Import the necessary packages to read the data
import pandas as pd
import numpy  as np # numpy is always useful
import os           # Loads the functions related to the operating system 

In [2]:
# SOLUTION
# Read the csv file with the training data - the index, i.e. the identifier of the roofs, is the first column (DF_UID)
data = pd.read_csv( 'training_sample.csv' , index_col = 0 )

## Step 1.2 - Correlation between variables

Using `pandas`, we can compute the correlation between each pair of variables. To improve the readability of the resulting table, we will style the table with red colors for positive values, and blue colors for negative values:

In [3]:
# Compute the correlation between variables
correlation_matrix = data.corr()
correlation_matrix.style.background_gradient(cmap='coolwarm', vmin = -1, vmax = 1).format("{:.2}") # Formatting

,roof_tilt,roof_aspect,roof_area,roof_x,roof_y,SIS,SISDIR,DHI,horizon_S,horizon_SSW,horizon_SWW,horizon_W,horizon_NWW,horizon_SSE,horizon_SEE,horizon_E,horizon_NEE,tilted_radiation
roof_tilt,1.0,0.025,-0.11,-0.056,0.13,-0.12,-0.14,0.044,0.11,0.061,-0.0059,-0.036,-0.097,0.1,0.031,0.047,0.031,-0.17
roof_aspect,0.025,1.0,0.0052,-0.045,0.016,0.0007,-0.021,0.048,-0.0016,-0.22,-0.29,-0.26,-0.13,0.052,0.31,0.37,0.35,-0.036
roof_area,-0.11,0.0052,1.0,-0.065,-0.028,0.011,0.025,-0.031,-0.021,-0.023,-0.043,-0.039,-0.034,-0.01,0.01,-0.0064,-0.017,0.028
roof_x,-0.056,-0.045,-0.065,1.0,0.23,0.065,-0.11,0.4,-0.11,-0.028,0.02,0.0083,0.02,-0.13,-0.11,-0.11,-0.12,0.0088
roof_y,0.13,0.016,-0.028,0.23,1.0,-0.76,-0.82,0.14,0.066,0.051,0.0053,-0.0051,-0.05,0.038,-0.012,-0.043,-0.022,-0.14
SIS,-0.12,0.0007,0.011,0.065,-0.76,1.0,0.9,0.2,-0.11,-0.072,0.005,0.011,0.059,-0.089,-0.032,0.0046,-0.023,0.22
SISDIR,-0.14,-0.021,0.025,-0.11,-0.82,0.9,1.0,-0.25,-0.087,-0.068,-0.023,-0.031,0.021,-0.059,-0.015,0.0087,-0.01,0.2
DHI,0.044,0.048,-0.031,0.4,0.14,0.2,-0.25,1.0,-0.042,-0.008,0.064,0.094,0.083,-0.066,-0.036,-0.0093,-0.027,0.033
horizon_S,0.11,-0.0016,-0.021,-0.11,0.066,-0.11,-0.087,-0.042,1.0,0.71,0.3,0.17,-0.073,0.9,0.55,0.2,0.14,-0.58
horizon_SSW,0.061,-0.22,-0.023,-0.028,0.051,-0.072,-0.068,-0.008,0.71,1.0,0.7,0.53,0.17,0.57,0.17,-0.086,-0.096,-0.48


## *Questions*
- a. The corrlation matrix shows a strong negative correlation between the y-coordinate of the roof (roof_y) and SIS/SISDIR. Can you explain why? Why do we see no such correlation for roof_x?
- b. The correlation matrix also shows two correlated "blocks" of the horizon values. What causes these blocks and what does that imply for the model?
- c. Which are the five variables with the strongest correlation to the target variable? Which are the five variables with the smallest correlation?

### *SOLUTION*
- a. The strong correlation is due to the fact that the y-coordinate represents the northern *latitude*, which determines the maximum solar angle and the total daylight hours; this influences the global and direct horizontal radiation - the further north we are, the lower the horizontal radiation. The roof_x is the *longitude*, which impacts the annual radiation to a much lesser extent.
- b. The two "blocks" are formed around the east and west horizon values, i.e. the horizon values for eastern azimuths are strongly positively correlated, and are slightly negatively correlated to the horizon values for western azimuths. The strong positive correlation suggests that we don't need *all* the horizon values, but a subset would suffice.
- c. The strongest correlation with the target is observed for the horizon values for **S, SSE, SSW, SEE and SWW**. The smallest correlation is seen for **roof_x, horizon_NWW, roof_area, DHI and roof_aspect**.<br>**NOTE**: Just because the 5 horizons have the highest correlation, it does not mean that these are the best features to select!

## Step 1.3 - Visualising features vs. targets 

To start, we set up the matplotlib interactive plotting:

In [4]:
%matplotlib notebook
import matplotlib.pyplot as plt

and define the list of features and the target, to know what to plot against each other:

In [5]:
target_name   = 'tilted_radiation'
features_list = [ 'roof_tilt', 'roof_aspect', 'roof_area', 'roof_x', 'roof_y', 'SIS',
                  'SISDIR', 'DHI', 'horizon_S', 'horizon_SSW', 'horizon_SWW', 'horizon_W',
                  'horizon_NWW', 'horizon_SSE', 'horizon_SEE', 'horizon_E', 'horizon_NEE',]

Now, we can create scatterplots of all features against the target:

In [6]:
# Create figure
fig, axes = plt.subplots(5,4, sharey = True, figsize = (10,10))
ax = axes.flatten()
for i, feature in enumerate(features_list):
    data.plot.scatter(x = feature, y = target_name, ax = ax[i], s = 2) # Plot feature against target
    # Format figure:
    ax[i].set_xlabel('')
    ax[i].set_title(feature)
    ax[i].grid(ls = '--')
    
    # For the roof area only, show the x-axis with logarithmic values:
    if feature == 'roof_area': ax[i].set_xscale('log')
for j in range(i+1, len(ax)): ax[j].axis('off') # deactivate unused sub-axes
plt.tight_layout()

<IPython.core.display.Javascript object>

## *Questions*
- a. Based on the scatterplots, which variable shows the strongest relatioship with the target?
- b. What is the correlation coefficient between this varialbe and the target?
- c. How do you explain the discrepancy, and what does this mean for the modelling?
- d. Can you think of a way to transform this variable in order to increase its correlation coefficient?
- e. For the other 4 features with the smallest correlation to the target from Step 1.2, do you observe a similar discrepancy or are the correlation coefficients confirmed by the plots?

### *SOLUTION*
- a. The scatterplots suggests that the tilted radiation depends strongly on the *__roof aspect__*. This does not come as a surprise; north-facing roofs receive less solar radiation than south-facing roofs.
- b. The correlation coefficient of the roof aspect is **-0.036**, making it one of the *lowest* coefficients in the dataset.
- c. The relationship of the roof_aspect to the tilted radiation is highly **non-linear** (rather quadratic). Consequently, the measure of *linear correlation* is low. This is an important pitfall of using correlation coefficients to measure the importance of features for the modelling. We should hence always cross-check the scatterplots.
- d. We could transform the roof aspect by dividing it into two features: *abs(roof_aspect)* and *sign(roof_aspect)*. This would help to *linearize* thia feature, and can significantly improve the performance of the linear regression that we have seen in Tutorial 1. Some estimators do not require linear features, but in some cases this kind of **feature transformation** can significantly improve model performance.
- e. In the other 4 cases (roof_x, horizon_NWW, roof_area, DHI), the low correlation coefficient is confirmed by a near-random scatterplot.

## Step 1.4 - Feature transformation
Let's transform the data in this way by computing first the sign and then replacing the roof aspect by it's absolute value:

In [7]:
# SOLUTION
data['aspect_sign'] = np.sign(data['roof_aspect']) ## Compute the sign of the roof_aspect
data['roof_aspect'] = abs(data['roof_aspect']) ## Compute the absolute value of the roof_aspect

We can check the correlation coefficient of the new features with the tilted radiation by using the `corrwith` function, and will see that it is much improved and that the roof_aspect becomes the *most important feature*:

In [8]:
data[['roof_aspect', 'aspect_sign']].corrwith(data['tilted_radiation'])

roof_aspect   -0.648147
aspect_sign   -0.037462
dtype: float64

Finally, we need to add the new feature `aspect_sign` to the feature list:

In [9]:
# SOLUTION
features_list = [ 'roof_tilt', 'roof_aspect', 'roof_area', 'roof_x', 'roof_y', 'aspect_sign', 'SIS',
                  'SISDIR', 'DHI', 'horizon_S', 'horizon_SSW', 'horizon_SWW', 'horizon_W',
                  'horizon_NWW', 'horizon_SSE', 'horizon_SEE', 'horizon_E', 'horizon_NEE',]

## Step 1.5 - Mutual information

In [10]:
from sklearn.feature_selection import mutual_info_regression

By splitting the data into target (y) and features (X), we can call the function and display it as a pandas dataframe:

In [11]:
# SOLUTION
y = data[ target_name ]   # Extract the target column
X = data[ features_list ] # Extract the list of all features 

In [12]:
(pd.DataFrame(mutual_info_regression(X, y), index = X.columns, columns = ['Mutual_info'])
     .sort_values('Mutual_info', ascending = False))

,Mutual_info
roof_aspect,0.605175
horizon_S,0.221153
roof_tilt,0.193899
horizon_SSW,0.172842
horizon_SSE,0.166318
SIS,0.066277
horizon_SEE,0.052925
aspect_sign,0.045374
SISDIR,0.038397
horizon_SWW,0.037294


## *Questions*
- a. Based on mutual information, which are the most important features?
- b. How does this answer change compared to the answers from Step 1.2?

### *SOLUTION*
- a. The most important features now are **roof aspect, horizon_S, roof_tilt, horizon_SSW and horizon_SSE**.
- b. The roof aspect and roof tilt have replaced the horizon values for SEE and SWW amongst the most important features. 


# Part 2 - Feature preprocessing and k-fold cross-validation<

The required packages are:

In [13]:
from sklearn.preprocessing   import scale
from sklearn.model_selection import KFold

from lib_file import *    # Functions in the file lib_file.py, which should be in the same directory as your notebook

## Step 2.1: k-fold cross-validation
Throughout this tutorial we will work with 5-fold cross-validation, which is the default in all of scikit's `cross_val`-functions:

In [14]:
N_FOLDS = 5 # Define the number of folds

We can declare a 5-fold cross-validation and plot the resulting splits for our data:

In [15]:
cv = KFold(n_splits = N_FOLDS)
cv

KFold(n_splits=5, random_state=None, shuffle=False)

In [16]:
plt.figure(figsize = (7, 3))
plot_cv_indices(cv, X, y, ax = plt.gca(), n_splits = N_FOLDS, cmap_cv = 'coolwarm')
plt.tight_layout()

<IPython.core.display.Javascript object>

## Step 2.2: Prepare features and target
To avoid having always the same (and potentially biased) subsets of the data in the cross-validation, let's shuffle the entire dataset randomly:

In [17]:
data_shuffled = data.sample( frac = 1 ) # Randomly sample from the entire dataset (i.e. reshuffle the data)

**NOTE**: This step introduces additional randomness in the modelling, and means that your results will vary every time you run the script.

Now, we need to extract the features and targets again, from the shuffled dataset:

In [18]:
# SOLUTION
y = data_shuffled[ target_name ]   # Extract the target column
X = data_shuffled[ features_list ] # Extract the list of features (later, we may reduce this list)

Finally, we should standardize the data using the `scale` function:

In [19]:
X_scaled = scale(X)

Later, we will need to know the total number of features, so let's assign this value to the variable `n_features_all`:

In [20]:
n_features_all = X_scaled.shape[1]
n_features_all

18

## Step 2.3: Save new datasets

In [21]:
data_shuffled.to_csv('training_shuffled.csv')

In [22]:
data_scaled = pd.concat([ pd.DataFrame(X_scaled, columns = X.columns, index = X.index), # Dataframe for X_scaled
                          y ], axis = 1)                                                # Append y horizontally

In [23]:
# SOLUTION
data_scaled.to_csv('training_scaled.csv')

# Part 3 - Feature selection using KNN

For the feature selection using KNN the following functions are needed:

In [24]:
from sklearn.feature_selection import SelectKBest          # The feature selection module
from sklearn.neighbors         import KNeighborsRegressor  # The KNN regression model
from sklearn.model_selection   import cross_val_score      # The cross-validation function

## Step 3.1: Feature selection - a first glance

In [25]:
# Set up a SelectKBest instance with mutual information as decision criterion to return the 8 best features
mutual_info = SelectKBest(mutual_info_regression, k = 8 )          # Initialise
X_reduced   = mutual_info.fit_transform(X_scaled, y)               # Extract the best features

In [26]:
knn      = KNeighborsRegressor() # Initiate model
cv_score = cross_val_score(knn, X_reduced, y, cv = N_FOLDS)
cv_score

array([0.69914982, 0.77101024, 0.72083147, 0.73852281, 0.76932203])

The `cv_score` shows the results for all folds, but often it is more informative to obtain the *mean* and *standard deviation* instead:

In [27]:
print( cv_score.mean() )
print( cv_score.std() )

0.7397672753774541
0.027783078270940394


##### Comparison with all features
As a comparison, let's look at the score obtained from the default knn with *all* features:

In [28]:
# Solution
knn      = KNeighborsRegressor() # Initiate model
cv_score = cross_val_score(knn, X_scaled, y, cv = N_FOLDS)

print( cv_score.mean() )
print( cv_score.std() )

0.6728433813934359
0.017495786135978953


##### Comparison with default feature set (before feature transformation)
Another comparison that we can perform is to the default dataset, i.e. before applying the feature transformation from Step 1.4. We have wrapped this analysis in the function `cross_val_score_default_knn()` (found in the lib_file.py). The default CV score (all features, no transformation) is:

In [29]:
cv_score = cross_val_score_default_knn()
print( cv_score.mean() )
print( cv_score.std() )

0.505666303468904
0.03045349442565308


## *Questions*
- a. Which performance metric is used in the `cross_val_score` method? What does a score of 1 represent? What about a score of 0?
- b. How is the performance of the model with the default feature set improved through feature transformation (with all features) and subsequently feature selection (keeping 8 features)?
- c. What does this very first glance suggest about the importance of feature engineering (transformation, selection, etc.)?

### *SOLUTION*
- a. The cross_val_score uses the default `score`-method of the passed algorithm, i.e. the R2-coefficient of determination, which characterises the model's goodness of fit (i.e. the percentage of variance explained). A score of 1 represents a perfect model, with all variance explained. A score of 0 (or near-zero) represents complete randomness.
- b. By feature transformation, the score is improved from around 0.5 to around 0.65, and by feature selection, it is further improved to above 0.75 - this is a significant improvement, **just by improving the data used for modelling, not by changing the model itself.**
- c. Feature engineering is at least as important as the modelling process itself!

## Step 3.2: Systematic feature selection

In [30]:
n_features = np.arange(1, n_features_all + 1) # Construct a list of all possible number of features
n_features

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18])

We next declare an empty dataframe that contains the k folds as rows, and the different number of features as columns:

In [31]:
cv_featureSelect = pd.DataFrame(index = range(N_FOLDS), columns = n_features).fillna(0)
cv_featureSelect.index.name   = 'CV iteration'
cv_featureSelect.columns.name = 'Number of features'

By iterating over all possible number of features, we compute the cross-validation scores for the default knn:

In [32]:
# Iterate over all possible numbers of features
for n in n_features: 
    # Extract the n best features
    mutual_info = SelectKBest(mutual_info_regression, k = n )
    X_reduced   = mutual_info.fit_transform(X_scaled, y)
    
    # Train a knn and score the performance with 5-fold cross-validation
    knn = KNeighborsRegressor() # Initiate model
    cv_featureSelect.loc[:,n] = cross_val_score(knn, X_reduced, y, cv = N_FOLDS)   

This is the resulting dataframe:

In [33]:
# Display the data
cv_featureSelect.round(3)

Number of features,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
CV iteration,,,,,,,,,,,,,,,,,,
0,0.527,0.515,0.637,0.687,0.674,0.694,0.711,0.715,0.719,0.688,0.671,0.663,0.659,0.639,0.638,0.666,0.658,0.657
1,0.459,0.664,0.751,0.770,0.741,0.796,0.757,0.778,0.772,0.785,0.788,0.765,0.723,0.716,0.718,0.685,0.685,0.679
2,0.518,0.653,0.755,0.773,0.792,0.766,0.754,0.766,0.726,0.736,0.735,0.721,0.703,0.696,0.706,0.669,0.655,0.657
3,0.533,0.601,0.773,0.788,0.762,0.779,0.778,0.772,0.768,0.771,0.750,0.720,0.729,0.751,0.741,0.732,0.730,0.704
4,0.571,0.623,0.773,0.816,0.803,0.826,0.780,0.800,0.771,0.785,0.756,0.720,0.690,0.681,0.675,0.668,0.684,0.668


**NOTE**: Since the data is randomly shuffled before splitting, the results will deviate between different runs and from the solution!

Let's compute, for each number of features, the mean and standard deviation, which we can visualise as a pandas dataframe:

In [34]:
# Obtain cross-validation statistics
cv_stats = pd.DataFrame( [ cv_featureSelect.mean().rename('mean'),    # compute mean across the k folds
                           cv_featureSelect.std() .rename('std') ] )  # compute std. deviaiton across the k folds

In [35]:
# Visualise dataframe with formatting
format_df( cv_stats, cmap='YlGn', decimals = 3)

Number of features,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
mean,0.522,0.611,0.738,0.767,0.754,0.772,0.756,0.766,0.751,0.753,0.74,0.718,0.701,0.697,0.696,0.684,0.682,0.673
std,0.0404,0.0594,0.0575,0.048,0.0513,0.0491,0.0277,0.0312,0.0265,0.0416,0.0429,0.036,0.0282,0.0417,0.0401,0.0277,0.03,0.0196


The performance of the KNN can also be visualised as a plot with errorbars:

In [36]:
plt.figure(figsize = (8,3))
cv_stats.loc['mean'].plot(yerr = cv_stats.loc['std'], ax = plt.gca())
plt.ylabel('Cross-validation score ($R^2$)')
plt.grid()
plt.tight_layout()

<IPython.core.display.Javascript object>

## *Questions*
- a. What is the optimal number of features? **NOTE**: These results can vary due to the re-shuffling of the data!
- b. How much of the improvement compared to the default model from tutorial 2 can be attributed to the feature transformation of the `roof_aspect`? How do you measure that?
- c. How much of the improvement can be attributed to the feature selection?

### *SOLUTION*
- a. The optimal number of features should lie somewhere between 4 - 9 features.
- b. If we compare the results with all features, we see that the score is increased from around 0.4 to 0.65, simply by transforming the `roof_aspect`.
- c. If we compare the score for the optimal number of features (around 0.75-0.8) to the score for all features, we see that the feature selection increases the average cv-score by another +0.1 approximately.